In [1]:
import import_ipynb
import sys
sys.path.append('C:\\Users\\USER\\JupyterProjects\\AdvLSTM')
from Stock_Dataset import StockDataset

importing Jupyter notebook from Stock_Dataset.ipynb
importing Jupyter notebook from feature_extract.ipynb


In [2]:
import os
print(os.getcwd())

C:\Users\USER\JupyterProjects\AdvLSTM


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import argparse
from Att_LSTM import attLSTM
import numpy as np
import time
from metric import metric_acc as ACC
from metric import metric_mcc as MCC
import matplotlib.pyplot as plt
import csv
import os

def train(attLSTM, lstm_optimizer,Partition, args): ## Data, loss function, argument
    trainloader = DataLoader(Partition['train'],
                             batch_size = args.batch_size,
                             shuffle=False, drop_last=True)

    attLSTM.train()

    train_loss = 0.0
    for i, (x,y) in enumerate(trainloader):
        
        lstm_optimizer.zero_grad()

        linear1 = nn.Linear(11,10)
        tanh1 = nn.Tanh()

        input_x = tanh1(linear1(x.float())).transpose(0,1).float().to(args.device)
        true_y = y.squeeze().float().to(args.device)

        attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]
        y_pred_lstm, attention_weight, attn_applied = attLSTM(input_x)

        # output_ = torch.where(output1 >= 0.5, 1.0, 0.0)
        # output_.requires_grad=True
        loss = args.loss_fn(y_pred_lstm, true_y)
        loss.backward()


        lstm_optimizer.step()## parameter 갱신

        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return attLSTM, train_loss


def validation(attLSTM, partition, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    attLSTM.eval()
    val_loss = 0.0

    with torch.no_grad():
        for i, (x, y) in enumerate(valloader):

            attLSTM.train()

            linear1 = nn.Linear(11, 10)
            tanh1 = nn.Tanh()

            input_x = tanh1(linear1(x.float())).transpose(0, 1).float().to(args.device)
            true_y = y.squeeze().float().to(args.device)

            attLSTM.hidden = [attLSTM.to(args.device) for hidden in attLSTM.init_hidden()]

            y_pred_lstm, attention_weight, attn_applied = attLSTM(input_x)

            loss = args.loss_fn(y_pred_lstm, true_y)

            val_loss += loss.item()

        val_loss = val_loss / len(valloader)
        return attLSTM, val_loss


def test(attLSTM,partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    attLSTM.eval()

    ACC_metric = 0.0
    MCC_metric = 0.0
    with torch.no_grad():
        for i, (x, y) in enumerate(testloader):

            # feature transform
            linear1 = nn.Linear(11, 10)
            tanh1 = nn.Tanh()

            input_x = tanh1(linear1(x.float())).transpose(0, 1).float().to(args.device)
            true_y = y.squeeze().float().to(args.device)

            attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]

            y_pred_lstm, attention_weight, attn_applied = attLSTM(input_x)

            output_ = torch.where(output1 >= 0.5, 1.0, 0.0)

            output_.requires_grad = True

            ACC_metric += ACC(output_, true_y)
            MCC_metric += MCC(output_, true_y)

        ACC_metric = ACC_metric / len(testloader)
        MCC_metric = MCC_metric / len(testloader)

        return ACC_metric, MCC_metric



def experiment(partition, args):
    attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                           args.dropout, args.use_bn, args.attention_head, args.attn_size,
                           activation="ReLU")

    attLSTM.to(args.device)

    if args.optim == 'SGD':
        lstm_optimizer = optim.SGD(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        lstm_optimizer = optim.RMSprop(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        lstm_optimizer = optim.Adam(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    # ===================================== #
    for epoch in range(args.epoch):
        ts = time.time()
        attLSTM, train_loss = train(attLSTM, lstm_optimizer, partition, args)

        attLSTM, val_loss = validation(attLSTM, partition, args)

        te = time.time()

        ## 각 에폭마다 모델을 저장하기 위한 코드
        torch.save(attLSTM.state_dict(), args.new_file_path + '\\' + str(epoch) +'_attLSTM' +'.pt')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch, train_loss, val_loss, te - ts))

    ## val_losses에서 가장 값이 최소인 위치를 저장함
    site_val_losses = val_losses.index(min(val_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴
    attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                           args.dropout, args.use_bn, args.attention_head, args.attn_size,
                           activation="ReLU")
    attLSTM.to(args.device)

    attLSTM.load_state_dict(torch.load(args.new_file_path + '\\' + str(site_val_losses) +'_attLSTM'+ '.pt'))

    ACC, MCC = test(attLSTM,partition, args)
    print('ACC: {}, MCC: {}'.format(ACC, MCC))

    with open(args.new_file_path + '\\'+ str(site_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
        print('ACC: {}, MCC: {}'.format(ACC, MCC), file=fd)

    result = {}

    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['ACC'] = ACC
    result['MCC'] = MCC

    return vars(args), result

importing Jupyter notebook from Att_LSTM.ipynb
importing Jupyter notebook from attention.ipynb
importing Jupyter notebook from metric.ipynb


In [4]:
# '^KS11' : KOSPI
# '^KQ11' : 코스닥
# '^IXIC' : 나스닥
# '^GSPC' : SNP 500 지수
# '^DJI' : 다우존수 산업지수
# '^HSI' : 홍콩 항생 지수
# '^N225' : 니케이지수
# '^GDAXI' : 독일 DAX
# '^FTSE' : 영국 FTSE
# '^FCHI' : 프랑스 CAC
# '^IBEX' : 스페인 IBEX
# '^TWII' : 대만 기권
# '^AEX' : 네덜란드 AEX
# '^BSESN' : 인도 센섹스
# 'RTSI.ME' : 러시아 RTXI
# '^BVSP' : 브라질 보베스파 지수
# 'GC=F' : 금 가격
# 'CL=F' : 원유 가격 (2000/ 8 / 20일 부터 데이터가 있음)
# 'BTC-USD' : 비트코인 암호화폐
# 'ETH-USD' : 이더리움 암호화폐

# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

# ========= experiment setting ========== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

args.save_file_path = "C:\\Users\\USER\\JupyterProjects\\AdvLSTM\\results"

## ======== data ============= #
# ['CHL', 'DCM' , 'DOW', 'NTT' , 'SYT', 'TOT'] 안되는거
# args.stock_list =  ['AAPL', 'AMZN', 'BA', 'BAC', 'BHP', 'BRK-B', 'CMCSA', 'CVX', 'D','DIS','DUK', 'EXC', 'GE', 'GOOGL', 'HD', 'INTC', 'JNJ', 'JPM', 'KO', 'MA', 'MMM', 'MO', 'MRK', 'MSFT', 'NGG','NVS', 'ORCL', 'PEP', 'PFE', 'PG', 'PTR', 'RDS-B', 'RIO', 'SO', 'SPY', 'T', 'TM', 'UNH', 'UPS', 'VALE', 'VZ', 'WFC', 'WMT', 'XOM']
args.stock_list =  ['AMZN']
#, 'BA', 'BAC']
args.data_count = len(args.stock_list)
args.train_start = "2007-01-03"
args.train_end = "2015-01-01"
args.validation_start = "2015-01-01"
args.validation_end = "2016-01-03"
args.test_start = "2016-01-04"
args.test_end = "2016-12-31"

# ====== hyperparameter ======= #
args.batch_size = 128
args.x_frames = 10
args.y_frames = 1
args.input_dim = 10
args.output_dim = 1
args.dropout = 0.15
args.use_bn = True
args.loss_fn = nn.BCELoss()  ## loss function for classification : cross entropy
args.optim = 'Adam'
args.lr = 0.0001
args.l2 = 0.00001 #?
args.epoch = 1
# ============= model ================== #
args.attLSTM = attLSTM

# ====== att_lstm hyperparameter ======= #
args.hid_dim = 10
args.attention_head = 1
args.attn_size = 10
args.num_layers = 1
args.attLSTM_x_frames = 1


In [5]:
## 실행 파일
with open(args.save_file_path + '\\' + 'AdvLSTM_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC", "avg_test_MCC"])

    for stock in args.stock_list:
        est = time.time()
        setattr(args, 'symbol', stock)
        args.new_file_path = args.save_file_path + '\\' + "AdvLSTM_" + args.symbol
        os.makedirs(args.new_file_path)

        # 0번째에 index 1번째에 stock 1개가 input으로 들어감
        trainset = StockDataset(args.symbol, args.x_frames, args.y_frames,
                                args.train_start, args.train_end)
        valset = StockDataset(args.symbol, args.x_frames, args.y_frames,
                              args.validation_start,args.validation_end)
        testset = StockDataset(args.symbol, args.x_frames, args.y_frames,
                               args.test_start, args.test_end)
        partition = {'train': trainset, 'val': valset, 'test': testset}


        setting, result = experiment(partition, args)
        eet = time.time()
        entire_exp_time = eet - est

        fig = plt.figure()
        plt.plot(result['train_losses'])
        plt.plot(result['val_losses'])
        plt.legend(['train_losses', 'val_losses'], fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('loss', fontsize=15)
        plt.grid()
        plt.savefig(args.new_file_path + '\\' + str(args.symbol) + '_fig' + '.png')
        plt.close(fig)

        # csv파일에 기록하기
        wr.writerow(["AdvLSTM", args.symbol,entire_exp_time, result['ACC'], result['MCC']])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
torch.Size([1, 128, 10])
torch.Size([1, 128, 10])
torch.Size([128, 20])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.